# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

Для начала импортируем всевозможные необходимые пакеты 

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import lightgbm as lgb
import sklearn
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder 
import lightgbm as lgb
from pymystem3 import Mystem
import re
import torch
from sklearn.metrics import f1_score, make_scorer
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords as nltk_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
import transformers
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.model_selection import TimeSeriesSplit
import time 
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier

Теперь импортируем файл с данными и проанализируем ео на наличие ошибок и аутлаеров

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
data.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Никаких странных значений нету, аутлаеров тоже не выявлено, данные понятный, ровные, ничего выбивающегося. А значит время делать сэмплирование

Единсвтенное замечание - это что у таргета очень мало негативный комментариев, всего 10%

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Молодец, исследован баланс классов. Это важная информация для задачи классификации.</div>

Также стоит сделать лемматизацию и очистку текста

In [6]:
text = list(data['text'])

In [7]:
corpus = []
for i in range(len(text)):
    r = re.sub('[^a-zA-Z]', ' ', text[i])
    r = r.lower()
    r = r.split()
    r = [word for word in r if word not in stopwords.words('english')]
    r = ' '.join(r)
    r = r.lower()
    corpus.append(r)
    
data['text'] = corpus

In [8]:
%%time

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data['text'] = data['text'].apply(lambda x: ' '.join([y.lemma_ for y in nlp(x)]))

CPU times: user 13min 51s, sys: 1.69 s, total: 13min 53s
Wall time: 13min 57s


In [9]:
data

,text,toxic
0,explanation edit make username hardcore metall...,0
1,aww match background colour seemingly stick th...,0
2,hey man really try edit war guy constantly rem...,0
3,make real suggestion improvement wonder sectio...,0
4,sir hero chance remember page,0
...,...,...
159446,second time ask view completely contradict cov...,0
159447,ashamed horrible thing put talk page,0
159448,spitzer umm there s actual article prostitutio...,0
159449,look like actually put speedy first version de...,0


Почитал про спейси, так действительно легче 

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
 👍 

Текст очищен, но я советую при чистке оставить символ ' (апостроф), он играет важную роль в английском языке.
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">

Твой код можно сделать оптимальнее. 
    
 1. Отказывайся от конструкций  типа 
    
        for i in range(len(corpus)):
    
    Гораздо по питоновски выглядит вот такой код:
        for sentence in corpus:
    
    Еще короче,  откзазываемся от "переделки" pd.Series в list и просто проитерируемся по pd.Series
    
        for sentence in data['text']:
    
 2. Но и это не самое короткое и оптимальное.  Вот такой вариант выглядит лучше и поднятнее:

    
         data['lemm_text'] = data['text'].apply(clear_text)
         data['lemm_text'] = data['lemm_text'].apply(lemmafunction)

    
Если операция выполняется долго, лучше добавить прогресс-бар.  Тогда используем progress_apply
    
    
    from tqdm.notebook import tqdm
    tqdm.pandas()

    data['lemm_text'] = data['text'].progress_apply(clear_text)
    data['lemm_text'] = data['lemm_text'].progress_apply(lemmafunction)
    
    
    
    
</div>



<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3"><b>образец комментария студента</b></font>

Я сначала сделал точь в точь как вы и написать во 2 пункте, 1 в 1 было,  но из за того что оно очень долго выполнлось, я решил попробовать так, так оно выполнилось гораздо быстрее

</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success"> 
 <b>v2</b>
Молодец, что используешь spacy, а также отлключил лишние операции. Это заметно ускоряет процесс лемматизации.

    
    
Кстати, есть еще один способ ускорить лемматизацию. 
    
Можно использовать spaCy pipeline (https://spacy.io/usage/processing-pipelines)
    
    
    lemm_texts = []
    total = data.shape[0]
    nlp_pipe = nlp.pipe(data['text'].values, disable = ['ner', 'parser'])
    
    for doc in tqdm(nlp_pipe, total=total):
        lemm_text = " ".join([token.lemma_ for token in doc])    
        lemm_texts.append(lemm_text) 
    
Когда я проверял, у меня получилось ускорить почти в два раза.</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Молодец, что используешь лемматизатор WordNetLemmatizer. Но в данном случае он отработал не очень хорошо, и если присмотреться к тексту это хорошо видно. Например, в в нулевой строке есть глагол made, а его начальная форма make. 
    
Как правило, совершаются следующие ошибки:
    
 - Лемматизация должна производиться по одному слову, а не весь комментарий целиком
 - Слова должны быть приведены к нижнему регистру
 - Кроме самого слова в лемматизатор нужно передать дополнительную информацию о части речи слова (POS тег). 
    
    
    
Ты можешь доработать подход с WordNetLemmatizer или использовать spaCy, там все отрабатывает "из коробки". Можешь посмотреть вот эту статью.  


https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/
    
Хочу обратить твое внимание, что код из вышеуказанной статьи выполняется несколько дольше, чем оптимизированный, вот из этого топика
    
https://stackoverflow.com/questions/50992974/nltk-wordnetlemmatizer-not-lemmatizing-as-expected    
    
    
    
Совет - старайся сразу проверять  результаты лемматизации. Например, для предложения
    
    sentence = "The striped bats are hanging on their feet for best"
    
После лемматизации должен получиться вот такой результат
    
    "the strip bat be hang on their foot for best"    
    
Если будешь лемматизировать по второму способу, то слово  striped может остаться без изменений, это тоже  нормально (особенность алгоритма).       
</div>

Теперь наши данные готовы к обучению

## Обучение

In [10]:
features = data['text']
target = data['toxic']
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=1)

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
    
Данные разделены на выборки. Но я бы посоветовал тебе изменить пропорции и выделить на тест 10%. Причина простая - модели, которые мы обучаем чувствительны к объему обучающих данных. Чем больше слов они увидят в процессе обучения, и оценят их вклад в "токсичность", тем лучше будут модели. А для корректного тестирования и 10% данных вполне достаточно, учитывая немаленький размер датасета.

</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Старайся не делать такие длинные строки. Стандарт PEP8 регулирует длину строки 79 символов. Придерживаться такого очень сложно, но лучше хотя-бы вмещать в экран, чтобы не приходилось использовать scroll.
    
    
А еще  если сомневаешься, как отформатировать правильно - всегда можно найти какой-нибудь оналйн-форматтер, например этот
    
  https://extendsclass.com/python-formatter.html
    
    
В качестве дополнительного развития могу порекомендоваь книгу "PYTHON  чистый код для продолжающих", Эл Свейгарт    
    
   
    
    
</div>

In [11]:
cv = CountVectorizer()
train_features = cv.fit_transform(train_features)
test_features = cv.transform(test_features)

In [13]:
grid = {
    'class_weight': ['balanced', None]
}
model_lr = LogisticRegression(random_state=12345)
lr_grid = GridSearchCV(model_lr,  param_grid=grid, scoring='f1')

lr_grid.fit(train_features, train_target)

best_score = lr_grid.best_score_
best_params = lr_grid.best_params_

print(best_score)
print(best_params)

0.7636424274650186
{'class_weight': None}


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
 Совет: линейные модели очень хорошо работают с признаками, полученными из текстов. С помощью TF-IDF мы получили очень длинные разряженные вектора. И действительно, они очень длинные (длина около 140 000), при этом очень мало значений отличаются от нуля. Так вот, линейные модели гораздо лучше деревьев справляются с такими признаками. Но есть один секрет. Нужно  немного ослабить регуляризацию у логистической регрессии, и сделать это можно подобрав гиперпараметр С (чем он больше тем слабее регуляризация). Оптимальное значение стоит поискать в диапазоне 5-15. </div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Не стоит раньше времени подглядывать в тестовую выборку. Вначале обучим все модели и выберем лучшую по оценкам на крос-валидации и только потом протестируем лучшую модель.</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3"><b>образец комментария студента</b></font>
   
Тоже сделаю через гридсерч, мне нравится как он работает, и только потом тогда подсмотрим в тестовую выборку

</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
<b>v2</b> 👍 </div>

Точность получилась хорошая, 0.76. Можно попробовать сделать ее лучше с помощью catboost'а

In [14]:
grid = {
    'depth': [6,12]
}
model_cat = CatBoostClassifier(random_state=12345, verbose = 25, iterations = 100)
model_cat = GridSearchCV(model_cat,  param_grid=grid,  scoring='f1')
model_cat.fit(train_features, train_target)
best_score = model_cat.best_score_ 
best_params = model_cat.best_params_
print(best_score)
print(best_params)

Learning rate set to 0.5
0:	learn: 0.3424968	total: 1.03s	remaining: 1m 41s
25:	learn: 0.1546922	total: 29.2s	remaining: 1m 22s
50:	learn: 0.1378414	total: 54.6s	remaining: 52.5s
75:	learn: 0.1288637	total: 1m 19s	remaining: 25.2s
99:	learn: 0.1217898	total: 1m 42s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3351465	total: 1.37s	remaining: 2m 15s
25:	learn: 0.1550014	total: 29.4s	remaining: 1m 23s
50:	learn: 0.1369883	total: 54.2s	remaining: 52s
75:	learn: 0.1275838	total: 1m 17s	remaining: 24.5s
99:	learn: 0.1221900	total: 1m 39s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3362582	total: 983ms	remaining: 1m 37s
25:	learn: 0.1550819	total: 26.3s	remaining: 1m 14s
50:	learn: 0.1364546	total: 50.7s	remaining: 48.8s
75:	learn: 0.1292381	total: 1m 15s	remaining: 23.7s
99:	learn: 0.1218034	total: 1m 38s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3437178	total: 1.36s	remaining: 2m 14s
25:	learn: 0.1557519	total: 28.1s	remaining: 1m 19s
50:	learn: 0.1381969	total: 53.

Точность получилась равно 0.74, хороший результат, но недостаточный, посмотрим на результаты тестовой выборки для линейной регрессии, так как она выдает результаты

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Что-то здесь совсем запутано. Похоже на неудачный копипаст из предыдущего проекта. 

 - зачем TimeSeriesSplit?
 - мы решаем задачу классификации, почему CatBoostRegressor   
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3"><b>образец комментария студента</b></font>
   
Я просто был очень горд собой когда написал это первый раз и теперь это везде сую :) Я только после отправки понял что я написал catboostregressor вместо classifier, и точно также с тайм сериес

</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
<b>v2</b> Сейчас все ОК</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
И вот сейчас,когда ты исследовал две (или больше) моделей нужно выбрать одну лучшую и провести её тестирование 
</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
<b>v2</b> 👍 </div>

In [19]:
predictions_test = lr_grid.best_estimator_.predict(test_features)
print(f1_score(test_target, predictions_test))

0.7594971303634873


Точность получилась выше 0.75, отличный результат!

## Выводы

По итогам было сделано следующее:

1. Импортированы и обработаны данные
2. Данные лемматизированы и очищены
3. Обучена и проверена модель обычной логистической регрессии, результат которой равен 0.76, результат неплохой
4. Обучена модель градиентного бустинга, результат неудовлетворительный

Модель линейной регрессии оказалось лучшей и равна 0.76 по результат ф1 тестирования

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий</b>
Давай подведем итоги. В целом с проектом ты справляешься - текст предобработан, извлечены признаки и обучены классификаторы. Достигнуто требуемое значение метрики f1.
    
Но кое с чем нужно еще поработать. 
    
 - Пожалуста доработай раздел с лемматизацией. Сейчас она выполняется некорректно.
 - Нельзя раньше времени подглядыать в тестовую выборку.
 - Пожалуйста исправь ошибки в ячейке с кэтбустом.   


     
Жду твоих исправлений :)
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> рекомендации по доп. материалам</b>
Если решишь погрузиться в область работы с текстами, очень советую несколько продвинутых бесплатных курсов.
    
   - Отличный бесплатный курс от Школы глубокого обучения МФТИ (https://stepik.org/org/dlschool), старт курса каждые пол года. Два семестра, один по основам и компьютерному зрению, второй по обработке естественного языка. Проходить нужно именно в таком порядке,т.к. почти весь современный NLP построен на нейронках.
    
   - "Нейронные сети и компьютерное зрение" от Samsung Research Russia (https://stepik.org/course/50352/syllabus). Есть также продолжение по NLP.  
   - Трек NLP от сообщества ODS https://ods.ai/tracks/nlp-course-autumn-22
    
    
А если на тебя произвели впечатление возможности ChatGPT и хочешь попробовать использовать возможности больших языковых моделей для решения своих задач, могу порекомендовать следующие курсы (но к сожалению все на английском).
    
  - https://www.coursera.org/learn/generative-ai-with-llms  (можно прослужать бесплатно)
  - Короткие курсы на сайте https://www.deeplearning.ai/short-courses/  Самый свежак - как писать промпты, как использовать LLM для создания собственных приложений (например как создать бота на основе ChatGPT, который будет отвечать на вопросы по внутренней документации компании).  
  - https://www.promptingguide.ai/introduction/settings
 
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3"><b>образец комментария студента</b></font>
   
Спасибо, но мне почему то очень тяжко дается работа с текстами, не знаю с чем это связано, оно мне совсем не нравится, остальное вот интересно, а тут почему то не очень.
Возможно я просто это плохо понял и не до конца разобрался, очень тяжелый топик.
Теория в этой части тоже почему то описана очень плохо :(
Но вам спасибо большое за доп материалы
Очень много времени ушло на переработку, спасибо еще раз за содействие
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v2</b>

Я тебя понял. Да, тексты вначале кажутся сложноватыми, слишком много новых слов "леммаизация", "токнизация", мого деталей, новых библиотек...
    
Но на мой взгляд это сейчас самая интересная и бурно развивающаяся тема в машинном обучении, и революция LLM совершается на наших глазах ))
    
Проект принят. Поздравляю и желаю дальнейших успехов!
</div>